# Recuperação da Informação e Busca na Web - 2018.1

### Atividade: Projeto Final - Agrupamento de Notícias
### Aluno: Johanny de Lucena Santos

## - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - 

In [106]:
import pandas as pd
import numpy as np

import nltk
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer

import re
import os
import codecs

import matplotlib.pyplot as plt
import sklearn.metrics as metrics

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.cluster import KMeans
from sklearn import feature_extraction
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.externals import joblib

import mpld3 #pip install mpld3

from __future__ import print_function


In [16]:
#nltk.download('punkt')

[nltk_data] Downloading package punkt to /home/johannydls/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

## Importando a base de dados necessária para o agrupamento de notícias
Aqui, é feita a importação do arquivo csv para a análise. Em seguida, substituo as linhas com NaN por um espaço em branco. E por fim, um array é criado, onde cada índice possui o título, subtítulo e conteúdo da notícia.

In [107]:
dataset = pd.read_csv('../data/estadao_noticias_eleicao.csv', encoding="utf-8")
dataset = dataset.replace(np.nan, '', regex=True)
documentos = dataset.titulo + " " + dataset.subTitulo + " " + dataset.conteudo

titulos = dataset.titulo
subtitulos = dataset.subTitulo
conteudo = dataset.conteudo

## Informações do dataset

In [108]:
print("%d notícias" % len(dataset))

8716 notícias


## Definindo número de clusters
Aqui, iremos calcular o número ideal de clusters para agrupar nossas notícias, baseado no número de colunas que nosso dataset possui.

In [109]:
labels  = dataset.columns
true_k = np.unique(labels).shape[0]

## Stopwords, stemming e tokenizing
* http://brandonrose.org/clustering
* http://brandonrose.org/top100

In [110]:
stopwords = nltk.corpus.stopwords.words('portuguese')
print (stopwords[:10])

['de', 'a', 'o', 'que', 'e', 'do', 'da', 'em', 'um', 'para']


In [95]:
stemmer = SnowballStemmer("portuguese")

### here I define a tokenizer and stemmer which returns the set of stems in the text that it is passed

In [111]:
# here I define a tokenizer and stemmer which returns the set of stems in the text that it is passed

def tokenize_and_stem(text):
    # first tokenize by sentence, then by word to ensure that punctuation is caught as it's own token
    tokens = [word for sent in nltk.sent_tokenize(text) for word in nltk.word_tokenize(sent)]
    filtered_tokens = []
    # filter out any tokens not containing letters (e.g., numeric tokens, raw punctuation)
    for token in tokens:
        if re.search('[a-zA-Z]', token):
            filtered_tokens.append(token)
    stems = [stemmer.stem(t) for t in filtered_tokens]
    return stems

In [112]:
def tokenize_only(text):
    # first tokenize by sentence, then by word to ensure that punctuation is caught as it's own token
    tokens = [word.lower() for sent in nltk.sent_tokenize(text) for word in nltk.word_tokenize(sent)]
    filtered_tokens = []
    # filter out any tokens not containing letters (e.g., numeric tokens, raw punctuation)
    for token in tokens:
        if re.search('[a-zA-Z]', token):
            filtered_tokens.append(token)
    return filtered_tokens

## Calculando TF-IDF
Através da biblioteca sklearn, calculamos o TF-IDF de cada palavra no documento que possui o conjunto de notícias

In [98]:
vectorizer = TfidfVectorizer(stop_words=stopwords,analyzer='word',ngram_range=(1, 1), lowercase=True, use_idf=True)
X = vectorizer.fit_transform(documentos)

## Utilizando o algoritmo K-Means
Aqui, usamos o algoritmo K-Means para gerar nosso modelo.

In [99]:
model = KMeans(n_clusters=true_k, init='k-means++',max_iter=100, n_init=1)
model.fit(X)

KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=100,
    n_clusters=6, n_init=1, n_jobs=1, precompute_distances='auto',
    random_state=None, tol=0.0001, verbose=0)

## Top 10 termos de cada cluster

In [148]:
order_centroids = model.cluster_centers_.argsort()[:,::-1]
terms = vectorizer.get_feature_names()

print("\n")

for i in range(true_k):
    print("\nCluster %d:" % (i+1), end='')
    for ind in order_centroids[i, :20]:
        print(' %s;' % terms[ind], end='')
    print()

print("\n")




Cluster 1: petrobrás; cpi; costa; diretor; estatal; youssef; ex; presidente; lava; jato; petrobras; roberto; doleiro; esquema; delação; federal; refinaria; comissão; paulo; operação;

Cluster 2: marina; psb; campos; aécio; candidata; dilma; disse; campanha; silva; ex; eduardo; turno; governo; rede; programa; candidato; pt; presidente; psdb; segundo;

Cluster 3: aécio; dilma; pmdb; pt; presidente; candidato; governo; campanha; psdb; governador; partido; disse; paulo; turno; ex; estado; senador; tucano; alckmin; apoio;

Cluster 4: fotos; acesse; galerias; especiais; estadão; eventos; veja; principais; mundo; dia; brasil; despudoradamente; despudor; desprovimento; desprezível; desprovidos; desprovido; desprovidas; desprivilegiada; desproporcionais;

Cluster 5: dilma; governo; presidente; disse; política; lula; brasil; afirmou; país; pt; aécio; sobre; reforma; ser; economia; inflação; vai; ainda; programa; segundo;

Cluster 6: federal; justiça; eleitoral; paulo; tribunal; presidente; se

In [87]:
totalvocab_stemmed = []
totalvocab_tokenized = []

for i in conteudo:
    allwords_stemmed = tokenize_and_stem(i)
    totalvocab_stemmed.extend(allwords_stemmed)
    
    allwords_tokenized = tokenize_only(i)
    totalvocab_tokenized.extend(allwords_tokenized)

In [88]:
vocab_frame = pd.DataFrame({'words': totalvocab_tokenized}, index = totalvocab_stemmed)
print ('Aqui têm ' + str(vocab_frame.shape[0]) + ' itens no vocab_frame')

Aqui têm 3385175 itens no vocab_frame


In [113]:
print(vocab_frame.head())

            words
brasíl   brasília
após         após
o               o
desg     desgaste
provoc  provocado


In [149]:
tfidf_vectorizer = TfidfVectorizer(max_df=0.8, max_features=200000,
                                 min_df=0.2, stop_words=stopwords,
                                 use_idf=True, tokenizer=tokenize_and_stem, ngram_range=(1,3))

tfidf_matrix = tfidf_vectorizer.fit_transform(documentos) #fit the vectorizer to synopses

print(tfidf_matrix.shape)

(8716, 113)


In [150]:
terms = tfidf_vectorizer.get_feature_names()

In [151]:
dist = 1 - cosine_similarity(tfidf_matrix)

In [152]:
km = KMeans(n_clusters=true_k)
km.fit(tfidf_matrix)
clusters = km.labels_.tolist()

In [153]:
joblib.dump(km,  'doc_cluster.pkl')

['doc_cluster.pkl']

In [154]:
km = joblib.load('doc_cluster.pkl')
clusters = km.labels_.tolist()

In [155]:
noticias = { 'titulo': titulos, 'subtitulo':subtitulos, 'conteudo':conteudo, 'cluster': clusters }
frame = pd.DataFrame(noticias, index = clusters, columns = ['titulo', 'subtitulo', 'conteudo', 'cluster'])

In [156]:
frame['cluster'].value_counts()

2    2189
4    2098
0    1599
3    1261
5    1209
1     360
Name: cluster, dtype: int64

In [158]:
print("Top termos por cluster:")
print()

order_centroids = km.cluster_centers_.argsort()[:, ::-1]

for i in range(true_k):
    print("Cluster %d palavras: " % i, end='')
    
    for ind in order_centroids[i, :6]:
        print(' %s' % vocab_frame.loc[terms[ind].split(' ')].values.tolist()[0][0].encode('utf-8', 'ignore'), end=',')
    
    print()
    print()
    
    print("Cluster %d titulos:" % i, end='')
    
    
    for titulo in frame.loc[i]['titulo'].values.tolist():
        print(' %s,' % titulo)
    
    print()
    print()

Top termos por cluster:

Cluster 0 palavras:  b'candidato', b'deputado', b'pelos', b'votos', b'governo', b'eleitoral',

Cluster 0 titulos: PT espera 30 mil pessoas em festa na Esplanada,
 PT espera 30 mil pessoas em festa na Esplanada,
 PT espera 30 mil pessoas em festa na Esplanada,
 PT espera 30 mil pessoas em festa na Esplanada,
 PT espera 30 mil pessoas em festa na Esplanada,
 PT espera 30 mil pessoas em festa na Esplanada,
 PT espera 30 mil pessoas em festa na Esplanada,
 PT espera 30 mil pessoas em festa na Esplanada,
 PT espera 30 mil pessoas em festa na Esplanada,
 PT espera 30 mil pessoas em festa na Esplanada,
 PT espera 30 mil pessoas em festa na Esplanada,
 PT espera 30 mil pessoas em festa na Esplanada,
 PT espera 30 mil pessoas em festa na Esplanada,
 PT espera 30 mil pessoas em festa na Esplanada,
 PT espera 30 mil pessoas em festa na Esplanada,
 PT espera 30 mil pessoas em festa na Esplanada,
 PT espera 30 mil pessoas em festa na Esplanada,
 PT espera 30 mil pessoas em 

 PT espera 30 mil pessoas em festa na Esplanada,
 PT espera 30 mil pessoas em festa na Esplanada,
 PT espera 30 mil pessoas em festa na Esplanada,
 PT espera 30 mil pessoas em festa na Esplanada,
 PT espera 30 mil pessoas em festa na Esplanada,
 PT espera 30 mil pessoas em festa na Esplanada,
 PT espera 30 mil pessoas em festa na Esplanada,
 PT espera 30 mil pessoas em festa na Esplanada,
 PT espera 30 mil pessoas em festa na Esplanada,
 PT espera 30 mil pessoas em festa na Esplanada,
 PT espera 30 mil pessoas em festa na Esplanada,
 PT espera 30 mil pessoas em festa na Esplanada,
 PT espera 30 mil pessoas em festa na Esplanada,
 PT espera 30 mil pessoas em festa na Esplanada,
 PT espera 30 mil pessoas em festa na Esplanada,
 PT espera 30 mil pessoas em festa na Esplanada,
 PT espera 30 mil pessoas em festa na Esplanada,
 PT espera 30 mil pessoas em festa na Esplanada,
 PT espera 30 mil pessoas em festa na Esplanada,
 PT espera 30 mil pessoas em festa na Esplanada,
 PT espera 30 mil pe

Cluster 2 palavras:  b'n\xc3\xa3o', b'pelos', b'p\xc3\xbablica', b'federal', b'ser', b'casos',

Cluster 2 titulos: Seis obstáculos e desafios do segundo mandato de Dilma,
 Seis obstáculos e desafios do segundo mandato de Dilma,
 Seis obstáculos e desafios do segundo mandato de Dilma,
 Seis obstáculos e desafios do segundo mandato de Dilma,
 Seis obstáculos e desafios do segundo mandato de Dilma,
 Seis obstáculos e desafios do segundo mandato de Dilma,
 Seis obstáculos e desafios do segundo mandato de Dilma,
 Seis obstáculos e desafios do segundo mandato de Dilma,
 Seis obstáculos e desafios do segundo mandato de Dilma,
 Seis obstáculos e desafios do segundo mandato de Dilma,
 Seis obstáculos e desafios do segundo mandato de Dilma,
 Seis obstáculos e desafios do segundo mandato de Dilma,
 Seis obstáculos e desafios do segundo mandato de Dilma,
 Seis obstáculos e desafios do segundo mandato de Dilma,
 Seis obstáculos e desafios do segundo mandato de Dilma,
 Seis obstáculos e desafios do 

 Seis obstáculos e desafios do segundo mandato de Dilma,
 Seis obstáculos e desafios do segundo mandato de Dilma,
 Seis obstáculos e desafios do segundo mandato de Dilma,
 Seis obstáculos e desafios do segundo mandato de Dilma,
 Seis obstáculos e desafios do segundo mandato de Dilma,
 Seis obstáculos e desafios do segundo mandato de Dilma,
 Seis obstáculos e desafios do segundo mandato de Dilma,
 Seis obstáculos e desafios do segundo mandato de Dilma,
 Seis obstáculos e desafios do segundo mandato de Dilma,
 Seis obstáculos e desafios do segundo mandato de Dilma,
 Seis obstáculos e desafios do segundo mandato de Dilma,
 Seis obstáculos e desafios do segundo mandato de Dilma,
 Seis obstáculos e desafios do segundo mandato de Dilma,
 Seis obstáculos e desafios do segundo mandato de Dilma,
 Seis obstáculos e desafios do segundo mandato de Dilma,
 Seis obstáculos e desafios do segundo mandato de Dilma,
 Seis obstáculos e desafios do segundo mandato de Dilma,
 Seis obstáculos e desafios do 

 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 

Cluster 5 palavras:  b'a\xc3\xa9cio', b'tucano', b'dilma', b'psdb', b'candidato', b'n\xc3\xa3o',

Cluster 5 titulos: ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,
 ,